In [3]:

%matplotlib inline
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import numpy as np
import os

import keras
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, Reshape
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam, RMSprop

import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.utils import np_utils
from keras.datasets import cifar10


Using TensorFlow backend.


In [10]:
#0-498 is test, 499-whatever is training
def splitDataset(xSet, ySet):
	xTrain = xSet[499:]
	xTest = xSet[:499]
	yTrain = ySet[499:]
	yTest = ySet[:499]

	return ((xTrain, yTrain), (xTest, yTest))


def resize_images(images): 
    new_images = []

    for i in images:
        new_images.append(np.array(cv2.resize(i, (224,224))))
    
    return new_images

def get_data():
    # Load the x and y datasets
    xSetGray = np.load("alphanum-hasy-data-X.npy", mmap_mode="r") # Must convert to RGB
    ySet = np.load("alphanum-hasy-data-y.npy", mmap_mode="r")

    # The x dataset we loaded is gray. However, VGG16 only accepts RGB values, so we need to convert it.
    xSet = []

    for i in xSetGray:
        xSet.append(cv2.cvtColor(i, cv2.COLOR_GRAY2BGR))

    (x_Train, y_Train), (x_Test, y_Test) = splitDataset(xSet, ySet)
   
    # One hot encode the values
    y_Test = keras.utils.to_categorical(y_Test)
    y_Train = keras.utils.to_categorical(y_Train)

    #Resize Images to be 224 x 224
    x_Train = resize_images(x_Train)
    x_Test = resize_images(x_Test)

    #Convert to np array
    x_Train = np.array(x_Train)
    x_Test = np.array(x_Test)
    
    return x_Train, y_Train, x_Test, y_Test

In [9]:
x_Train, y_Train, x_Test, y_Test = get_data()

In [5]:
model=  VGG16(include_top=True, weights='imagenet')
model.compile('sgd','mse')
#model.summary()
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 116
batch_size = 20
epochs= 10


In [106]:
datagen_train = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen_test = ImageDataGenerator(rescale=1./255)

datagen_train.fit(x_Train)
datagen_train.fit(x_Test)
generator_train = datagen_train.flow(x=x_Train, y=y_Train, batch_size=batch_size)
generator_test = datagen_test.flow(x=x_Test, y=y_Test, batch_size=batch_size)


MemoryError: 

In [107]:
def predict(image_path):
    # Load and resize the image using PIL.
    img = PIL.Image.open(image_path)
    img_resized = img.resize(input_shape, PIL.Image.LANCZOS)

    # Plot the image.
    plt.imshow(img_resized)
    plt.show()

    # Convert the PIL image to a numpy-array with the proper shape.
    img_array = np.expand_dims(np.array(img_resized), axis=0)

    # Use the VGG16 model to make a prediction.
    # This outputs an array with 1000 numbers corresponding to
    # the classes of the ImageNet-dataset.
    pred = model.predict(img_array)
    
    # Decode the output of the VGG16 model.
    pred_decoded = decode_predictions(pred)[0]

    # Print the predictions.
    for code, name, score in pred_decoded:
        print("{0:>6.2%} : {1}".format(score, name))

In [109]:
transfer_layer = model.get_layer('block5_pool')

In [110]:
transfer_layer.output
conv_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [134]:
# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add the final layer for the actual classification.
new_model.add(Dense(116, activation='softmax'))

optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

In [135]:
def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [136]:
conv_model.trainable = False
for layer in conv_model.layers:
    layer.trainable = False


In [137]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [138]:
epochs = 20
steps_per_epoch = 100
steps_test = 26.5


In [ ]:
training = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/20
